## Project Name: Dementia Probabilities in HRS

Project Synopsis: Follows Hurd's (2013) method of dementia probability 
assignment using separate ordered probits for self- and proxy-respondents on the 
ADAMS Wave A cohort as described in the online appendix.  Recreates tables from 
the appendix and examines sensitivity/specificity and AUC of model vs. Hurd's 
publicly available probabilities in manner of Gianattasio, et al. (2019).  Final 
probabilities are correlated with Hurd’s at .96, with slightly higher sensitivity 
and lower specificity, and marginally higher AUC in and out of sample than
Hurd’s original probabilities.<br>

Principal Investigator: Amy Kelley <br> <br>
Created By: Evan Bollens-Lund<br>
Date Created: 12/10/15<br>
Last Modified By: Evan Bollens-Lund<br>
Date Last Modified: 2/25/19<br>

Data: HRS Core public files, 1998-2014
Software: SAS and Stata

Papers that use data derived from this code:
Ornstein, K.A., Garrido, M.M., Siu, A.L., Bollens‐Lund, E., Langa, K.M. and Kelley, A.S., 2018. Impact of In‐Hospital Death on Spending for Bereaved Spouses. Health Services Research, 53, pp.2696-2717.

Ornstein, K.A., Garrido, M.M., Siu, A.L., Bollens-Lund, E., Rahman, O.K. and Kelley, A.S., 2019. An Examination of Downstream Effects of Bereavement on Healthcare Utilization for Surviving Spouses in a National Sample of Older Adults. Forthcoming from PharmacoEconomics.

Wachterman, M.W., O’Hare, A.M., Rahman,O.K., Lorenz, K.A., Marcantonio, E.R., Alicante, G.K. and Kelley, A.S., 2019. Framing Prognostic Expectations: One-year Mortality After Dialysis Initiation Among Older Adults.  Forthcoming from JAMA Internal Medicine.

Using the original Hurd probabilities:
Kelley, A.S., McGarry, K., Gorges, R. and Skinner, J.S., 2015. The burden of health care costs for patients with dementia in the last 5 years of life. Annals of internal medicine, 163(10), pp.729-736.

Reference papers: 
Hurd, Michael D., Paco Martorell, Adeline Delavande, Kathleen J. Mullen, 
 and Kenneth M. Langa. "Monetary costs of dementia in the United States." 
 New England Journal of Medicine 368, no. 14 (2013): 1326-1334.

Gianattasio, Kan Z., Qiong Wu, M. Maria Glymour, and Melinda C. Power. "
Comparison of methods for algorithmic classification of dementia status 
 in the Health and Retirement Study." Epidemiology 30, no. 2 (2019): 291-302.


## SAS

In [ ]:
= V4 Outline MultiLine NoSorting TabWidth=30
H="sas get tics results"
libname cogn 'E:\data\hrs_public_2014\HRS_Cog';

proc import datafile='E:\data\hrs_public_2014\HRS_Cog\COGIMP9214A_R.dta' out=cog replace; run;

%macro gettics(w=,year=);

data tics1_&w;
set cog;
keep hhid pn core_year prediction_year
r&w.cogtot r&w.status r&w.bwc20 r&w.vp
 r&w.ser7  r&w.scis r&w.cact r&w.pres r&w.mo r&w.dy r&w.yr r&w.dw
 r&w.imrc r&w.dlrc r&w.vocab r&w.mstot 
 r&w.fbwc20 r&w.fvp
 r&w.fser7  r&w.fscis r&w.fcact r&w.fpres r&w.fmo r&w.fdy r&w.fyr r&w.fdw
 r&w.fimrc r&w.fdlrc r&w.fvocab;
core_year=&year;
prediction_year=&year+1;
run;

data tics2_&w;
set tics1_&w;
rename r&w.cogtot=TICS_tot;
rename r&w.status=imp_elig_status;
rename r&w.mo=mo;
rename r&w.dy=dy;
rename r&w.yr=yr;
rename r&w.dw=dw;
rename r&w.bwc20=bwc20;
rename r&w.ser7=ser7 ;
rename r&w.scis=scis;
rename r&w.cact=cact;
rename r&w.pres=pres;
rename r&w.imrc=imrc;
rename r&w.dlrc=dlrc;
rename r&w.vp=vp;
rename r&w.vocab=vocab;
rename r&w.mstot=mstot;

rename r&w.fmo=mo_imp;
rename r&w.fdy=dy_imp;
rename r&w.fyr=yr_imp;
rename r&w.fdw=dw_imp;
rename r&w.fbwc20=bwc20_imp;
rename r&w.fser7=ser7_imp;
rename r&w.fscis=scis_imp;
rename r&w.fcact=cact_imp;
rename r&w.fpres=pres_imp;
rename r&w.fimrc=imrc_imp;
rename r&w.fdlrc=dlrc_imp;
rename r&w.fvp=vp_imp;
rename r&w.fvocab=vocab_imp;




%mend;

%gettics(w=4,year=1998);
%gettics(w=5,year=2000);
%gettics(w=6,year=2002);
%gettics(w=7,year=2004);
%gettics(w=8,year=2006);
%gettics(w=9,year=2008);
%gettics(w=10,year=2010);
%gettics(w=11,year=2012);
%gettics(w=12,year=2014);

data tics_9814;
set tics2_4 tics2_5 tics2_6 tics2_7 tics2_8 tics2_9 tics2_10 tics2_11 tics2_12;
label TICS_tot='TICS Cognition Score, scale 0-35';
run;

proc sort nodupkey; by hhid pn core_year ; run;

/*convert to stata*/
proc export data=tics_9814 
outfile="E:\data\hrs_cleaned\working\tics_9814.dta"
replace;
run;


H="Proxies"
libname raw 'E:\data\hrs_public_2014\core';

proc import datafile="E:\data\hrs_public_2012\core\raw\H98PC_R.dta" out=h98pc_r replace;
run;

data core1998a;
set h98pc_r;
keep hhid pn f1389-f1460;
run;

data core1998;
set core1998a;
core_year=1998;
run;

data core2000;
set raw.core2000;
keep core_year hhid pn g1543-g1614;
core_year=2000;
run;

data core2002;
set raw.core2002;
keep core_year hhid pn hd506-hd554;
core_year=2002;
run;

data core2004;
set raw.core2004;
keep core_year hhid pn jd506-jd554;
core_year=2004;
run;

data core2006;
set raw.core2006;
keep core_year hhid pn kd506-kd554;
core_year=2006;
run;

data core2008;
set raw.core2008;
keep core_year hhid pn ld506-ld554;
core_year=2008;
run;

data core2010;
set raw.core2010;
keep core_year hhid pn md506-md554;
core_year=2010;
run;

data core2012;
set raw.core2012;
keep core_year hhid pn nd506-nd554;
core_year=2012;
run;


data core2014;
set raw.core2014;
keep core_year hhid pn od506-od554;
core_year=2014;
run;

data proxy;
set core1998 core2000 core2002 core2004 core2006 core2008 core2010 core2012 core2014;
run;

proc export data=proxy outfile="E:\data\serious_ill\int_data\proxy.dta" replace; run;


H="Proxy stata"